# Лабораторная работа 1: Реккурентные нейронные сети и слой Embedding

**Выполнили:** Алёшина Арина, Базылева Алена (15ФПЛ)

## Задача 2: Генерация текстов

In [12]:
from keras.layers.recurrent import LSTM, GRU
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize
import math

В качестве текстов для обучения генератора были выбраны книги С. Лукьяненко "Ночной дозор" и "Дневной дозор"

In [13]:
INPUT_FILE = "1_nochnoy_dozor.txt"
TEST_FILE = "2_dnevnoy_dozor.txt"

# extract the input as a stream of characters
print("Extracting text from input...")
file1 = open(INPUT_FILE, 'r', encoding='utf-8')
lines1 = []
for line in file1:
    line = line.strip().lower()
    if len(line) == 0:
        continue
    lines1.append(line)
file1.close()
text_1 = " ".join(lines1)
text_1 = re.sub('\ufeff', '', text_1)

file2 = open(TEST_FILE, 'r', encoding='utf-8')
lines2 = []
for line in file2:
    line = line.strip().lower()
    if len(line) == 0:
        continue
    lines2.append(line)
file2.close()
text_2 = " ".join(lines2)
text_2 = re.sub('\ufeff', '', text_2)

Extracting text from input...


In [14]:
train_tokens = [tok for tok in word_tokenize(text_1) if tok.isalpha()][:10000]
test_tokens_all = [tok for tok in word_tokenize(text_2) if tok.isalpha()][:10000]
test_tokens = [tok for tok in test_tokens_all if tok in train_tokens]  #выкидываем новые слова из тестового корпуса

Уменьшение кол-ва входных данных сделано с целью предотвращения ошибки MemoryError на этапе векторизации данных. Изначально планировалось использовать метод генераторов и функции fit_generator (и аналогичные), но при их использовании jupyter сильно зависает и в конце-концов отключается. Поэтому был применен несколько "варварский" метод сокращения объема данных.

In [15]:
chars = list(set(train_tokens))
nb_chars = len(chars)
char2index = dict((c,i) for i, c in enumerate(chars))
index2char = dict((i,c) for i, c in enumerate(chars))

Для предсказания был выбран размер последовательности 4, т.к. согласно некоторым исследованиям (потеряли ссылку, просим прощения), n-граммы из 5 слов хорошо предсказываются моделями

In [16]:
SEQLEN = 4
STEP = 1

In [17]:
train_chars = []
train_labels = []
test_chars = []
test_labels = []
for i in range(0, len(train_tokens) - SEQLEN, STEP):
    train_chars.append(train_tokens[i:i + SEQLEN])
    train_labels.append(train_tokens[i + SEQLEN])
for j in range(0, len(test_tokens) - SEQLEN, STEP):
    test_chars.append(test_tokens[j:j + SEQLEN])
    test_labels.append(test_tokens[j + SEQLEN])

In [18]:
X = np.zeros((len(train_chars), SEQLEN))
y = np.zeros((len(train_chars), nb_chars))
for i, input_char in enumerate(train_chars):
    for j, ch in enumerate(input_char):
        X[i, j] = char2index[ch]
    y[i, char2index[train_labels[i]]] = 1
Xtrain, Xval, Ytrain, Yval = train_test_split(X, y, test_size=0.2)

Функции для подсчета перплексии и качества работы модели

In [19]:
def perplexity(probabilities):
    sumlog = 0
    for probability in probabilities:
        log_prob = math.log2(probability)
        sumlog += log_prob
    l = sumlog/len(probabilities)
    perp = math.pow(2, -l)
    return perp

In [31]:
def eval_model(model):
    example = []
    for i in range(len(test_chars)):
        probabilities = []
        test_input = test_chars[i]
        test_l = test_labels[i]
        test_s = np.zeros((1, SEQLEN))
        for w, word in enumerate(test_chars):
            test_s[0, w] = char2index[word]
        pred = model.predict(test_s, verbose=0)[0]
        probability = pred[char2index[test_l]]
        probabilities.append(probability)
    perp = perplexity(probabilities)
    return perp

### LSTM

In [21]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(Embedding(nb_chars, 100, input_length=SEQLEN))
model.add(LSTM(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_chars),
                    unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 4, 100)            407000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_4 (Dense)              (None, 4070)              525030    
_________________________________________________________________
activation_4 (Activation)    (None, 4070)              0         
Total params: 1,049,278
Trainable params: 1,049,278
Non-trainable params: 0
_________________________________________________________________


Эксперименты с размерами слоя Embedding показали, что при увеличении этого показателя (200 и далее) качество генерируемого текста несколько хуже, в то время как сильное его уменьшение порождает последовательность 5-7 однотипных предлогов или местоимений

In [22]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(Xtrain, Ytrain, validation_data=[Xval, Yval], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    test_idx = np.random.randint(len(train_chars))
    test_chars = train_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(' '.join(test_chars))
    for i in range(NUM_PREDS_PER_EPOCH):
        test_seq = np.zeros((1, SEQLEN))
        for i, ch in enumerate(test_chars):
            test_seq[0, i] = char2index[ch]
        pred = model.predict(test_seq, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end=" ")
        test_chars = test_chars[1:] + [ypred]
    print()

Iteration #: 0
Train on 7996 samples, validate on 2000 samples
Epoch 1/1
7996/7996 [==============================] - 10s 1ms/step - loss: 7.7833 - acc: 0.0259 - val_loss: 7.6046 - val_acc: 0.0290
Generating from seed: ['убил', 'бы', 'меня', 'инициировал']
убил бы меня инициировал
не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не 
Iteration #: 1
Train on 7996 samples, validate on 2000 samples
Epoch 1/1
7996/7996 [==============================] - 7s 879us/step - loss: 7.2448 - acc: 0.0279 - val_loss: 7.8667 - val_acc: 0.0225
Generating from seed: ['ваше', 'благородие', 'не', 'рад']
ваше благородие не рад
в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в 

In [32]:
print(eval_model(model))

578312.1447032659


> Следует сказать, что сгенерированный текст выглядит неплохо для такого малого объема входных данных. Видно, что в какой-то момент генератор "сбивается" и начинает повторяться, но начальные строки в целом выглядят интересно.

## GRU

In [33]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 40
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model_1 = Sequential()
model_1.add(Embedding(nb_chars, 100, input_length=SEQLEN))
model_1.add(GRU(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_chars),
                    unroll=True))
model_1.add(Dense(nb_chars))
model_1.add(Activation("softmax"))

model_1.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['acc'])
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 4, 100)            407000    
_________________________________________________________________
gru_4 (GRU)                  (None, 128)               87936     
_________________________________________________________________
dense_5 (Dense)              (None, 4070)              525030    
_________________________________________________________________
activation_5 (Activation)    (None, 4070)              0         
Total params: 1,019,966
Trainable params: 1,019,966
Non-trainable params: 0
_________________________________________________________________


In [34]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model_1.fit(Xtrain, Ytrain, validation_data=[Xval, Yval], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    test_idx = np.random.randint(len(train_chars))
    test_chars = train_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(' '.join(test_chars))
    for i in range(NUM_PREDS_PER_EPOCH):
        test_seq = np.zeros((1, SEQLEN))
        for i, ch in enumerate(test_chars):
            test_seq[0, i] = char2index[ch]
        pred = model_1.predict(test_seq, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end=" ")
        test_chars = test_chars[1:] + [ypred]
    print()

Iteration #: 0
Train on 7996 samples, validate on 2000 samples
Epoch 1/1
7996/7996 [==============================] - 10s 1ms/step - loss: 7.7786 - acc: 0.0245 - val_loss: 7.6076 - val_acc: 0.0290
Generating from seed: ['попытаться', 'но', 'не', 'сейчас']
попытаться но не сейчас
не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не 
Iteration #: 1
Train on 7996 samples, validate on 2000 samples
Epoch 1/1
7996/7996 [==============================] - 7s 938us/step - loss: 7.2475 - acc: 0.0269 - val_loss: 7.8732 - val_acc: 0.0285
Generating from seed: ['неделя', 'подготовки', 'не', 'прошла']
неделя подготовки не прошла
я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я 

In [35]:
print(eval_model(model_1))

163095.09653006427


In [36]:
HIDDEN_SIZE = 128
BATCH_SIZE = 64
NUM_ITERATIONS = 40
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model_1 = Sequential()
model_1.add(Embedding(nb_chars, 100, input_length=SEQLEN))
model_1.add(GRU(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_chars),
                    unroll=True))
model_1.add(Dense(nb_chars))
model_1.add(Activation("softmax"))

model_1.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['acc'])
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 4, 100)            407000    
_________________________________________________________________
gru_5 (GRU)                  (None, 128)               87936     
_________________________________________________________________
dense_6 (Dense)              (None, 4070)              525030    
_________________________________________________________________
activation_6 (Activation)    (None, 4070)              0         
Total params: 1,019,966
Trainable params: 1,019,966
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model_1.fit(Xtrain, Ytrain, validation_data=[Xval, Yval], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    test_idx = np.random.randint(len(train_chars))
    test_chars = train_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(' '.join(test_chars))
    for i in range(NUM_PREDS_PER_EPOCH):
        test_seq = np.zeros((1, SEQLEN))
        for i, ch in enumerate(test_chars):
            test_seq[0, i] = char2index[ch]
        pred = model_1.predict(test_seq, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end=" ")
        test_chars = test_chars[1:] + [ypred]
    print()

Iteration #: 0
Train on 7996 samples, validate on 2000 samples
Epoch 1/1
7996/7996 [==============================] - 12s 1ms/step - loss: 7.6902 - acc: 0.0274 - val_loss: 7.6552 - val_acc: 0.0225
Generating from seed: ['это', 'время', 'придется', 'найти']
это время придется найти
в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в в 
Iteration #: 1
Train on 7996 samples, validate on 2000 samples
Epoch 1/1
7996/7996 [==============================] - 10s 1ms/step - loss: 7.2793 - acc: 0.0281 - val_loss: 7.8652 - val_acc: 0.0285
Generating from seed: ['достаточно', 'компактном', 'еще', 'плеер']
достаточно компактном еще плеер
я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я 
Iteration #: 2
Train

In [ ]:
print(eval_model(model_1))

In [11]:
HIDDEN_SIZE = 128
BATCH_SIZE = 64
NUM_ITERATIONS = 30
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100
SEQLEN = 10  #попробуем с бОльшим размером последовательности

model_1 = Sequential()
model_1.add(Embedding(nb_chars, 100, input_length=SEQLEN))
model_1.add(GRU(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_chars),
                    unroll=True))
model_1.add(Dense(nb_chars))
model_1.add(Activation("softmax"))

model_1.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['acc'])
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 100)           407000    
_________________________________________________________________
gru_3 (GRU)                  (None, 128)               87936     
_________________________________________________________________
dense_3 (Dense)              (None, 4070)              525030    
_________________________________________________________________
activation_3 (Activation)    (None, 4070)              0         
Total params: 1,019,966
Trainable params: 1,019,966
Non-trainable params: 0
_________________________________________________________________


In [9]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model_1.fit(Xtrain, Ytrain, validation_data=[Xval, Yval], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    test_idx = np.random.randint(len(train_chars))
    test_chars = train_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(' '.join(test_chars))
    for i in range(NUM_PREDS_PER_EPOCH):
        test_seq = np.zeros((1, SEQLEN))
        for i, ch in enumerate(test_chars):
            test_seq[0, i] = char2index[ch]
        pred = model_1.predict(test_seq, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end=" ")
        test_chars = test_chars[1:] + [ypred]
    print()

Iteration #: 0
Train on 7992 samples, validate on 1998 samples
Epoch 1/1
7992/7992 [==============================] - 14s 2ms/step - loss: 7.6918 - acc: 0.0268 - val_loss: 7.6981 - val_acc: 0.0295
Generating from seed: ['отвечаю', 'быстро', 'сказал', 'я', 'обрывая', 'связь', 'и', 'остановился', 'перед', 'милицейским']
отвечаю быстро сказал я обрывая связь и остановился перед милицейским
я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я я 
Iteration #: 1
Train on 7992 samples, validate on 1998 samples
Epoch 1/1
7992/7992 [==============================] - 12s 1ms/step - loss: 7.3006 - acc: 0.0262 - val_loss: 7.8791 - val_acc: 0.0295
Generating from seed: ['лилась', 'музыка', 'тихая', 'едва', 'слышная', 'но', 'удивительно', 'приятная', 'тонкое', 'пение']
лилась музыка тихая едва слышная но удивительно приятная тонкое пение
я я я я я я я я я я

Возможно, бОльшее кол-во итераций могло бы улучшить предсказательную способность модели, но компьютер потянул максимум 30 итераций, на большем кол-ве он зависал посередине.  
Как видно, увеличение SEQLEN не сильно повлияло на результат, однако уменьшение размера батча несколько улучшило сгенерированный текст